# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

## Github Repository: <https://github.com/jonathan-cleave/GSB-544-.git>

In [2]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [3]:
# YOUR CODE HERE
URL = ("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [4]:
# YOUR CODE HERE
soup = BeautifulSoup(response.text, "html.parser")
soup.head()

[<meta charset="utf-8"/>,
 <title>List of United States cities by population - Wikipedia</title>,
 <script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available";var cookie=document.cookie.match(/(?:^|; )enwikimwclientpreferences=([^;]+)/);if(cookie){cookie[1].split('%2C').forEach(function(pref){className=className.replace(new RegExp('(^| )'+pref.replace(/-clientpref-\w+$|[^\w-]+/g,'')+'-clientpref-\\w+( |$)'),'$1'+pref+'$2');});}document.documentElement.className=className;}());RLCONF={"wgBreakFrames":fals

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [6]:
tables = soup.find_all("table")
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [7]:
len(soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  ))

1

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [8]:
table = soup.find_all("table",
                  attrs={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"}
                  )[0]


**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [9]:
cells = table.find_all("tr")
len(cells)

city = table.find_all("a")[0]
city
state = table.find_all("")[1]
state
pop = table.find_all("a")[2]
area = table.find_all("a")[5]

city_name = city.get_text(strip=True)
city_name
state_name = state.get_text(strip=True)
state_name
pop_name = pop.get_text(strip=True)
pop_name
area_name = area.get_text(strip=True)
area_name

IndexError: list index out of range

In [10]:
rows = []

for row in table.find_all("tr")[1:]:
    cells = row.find_all("td")
    if len(cells) > 5:  # Ensure there are enough columns to extract data
        city = cells[0].find("a")
        city_name = city.get_text(strip=True) if city else ""

        state = cells[1].find("a")
        state_name = state.get_text(strip=True) if state else ""

        pop = cells[2]
        pop_name = pop.get_text(strip=True) if pop else ""

        area = cells[5]
        area_name = area.get_text(strip=True) if area else ""


        rows.append({
            "city": city_name,
            "state": state_name,
            "pop": pop_name,
            "land area (mi^2)": area_name
        })


In [ ]:
pd.DataFrame(rows)

,city,state,pop,land area (mi^2)
0,New York,NY,"8,478,072",300.5
1,Los Angeles,CA,"3,878,704",469.5
2,Chicago,IL,"2,721,308",227.7
3,Houston,TX,"2,390,125",640.4
4,Phoenix,AZ,"1,673,164",518.0
...,...,...,...,...
341,Deltona,FL,"100,513",37.3
342,Federal Way,WA,"100,252",22.3
343,San Angelo,TX,"100,159",59.7
344,Tracy,CA,"100,136",25.9


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [11]:
URL = ("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [14]:
df = pd.read_html(response.text, attrs ={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"})
len(df)

/tmp/ipython-input-1344781294.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(response.text, attrs ={


1

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [15]:
df_cities2 = pd.read_html(response.text, attrs ={
                      "class": "sortable wikitable sticky-header-multi static-row-numbers sort-under col1left col2center",
                      "style": "text-align:right"})[0]
df_cities2

/tmp/ipython-input-1257916110.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_cities2 = pd.read_html(response.text, attrs ={


City  ST 2024 estimate 2020 census  Change 2020 land area          \
            City  ST 2024 estimate 2020 census  Change            mi2     km2   
0    New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
1    Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
2        Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
3        Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
4        Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   
..           ...  ..           ...         ...     ...            ...     ...   
341      Deltona  FL        100513       93692  +7.28%           37.3    96.6   
342  Federal Way  WA        100252      101030  −0.77%           22.3    57.8   
343   San Angelo  TX        100159       99893  +0.27%           59.7   154.6   
344        Tracy  CA        100136       93000  +7.67%           25.9    67.1   
345      Sunrise  FL        100128       97335  +2.87%           16.2    42.0   

    2020 density                                      Location  
           / mi2  / km2                               Location  
0          29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W  
1           8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W  
2          12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W  
3           3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W  
4           3105   1199  33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W  
..           ...    ...                                    ...  
341         2512    970    28°55′N 81°13′W﻿ / ﻿28.91°N 81.21°W  
342         4530   1750  47°19′N 122°20′W﻿ / ﻿47.31°N 122.34°W  
343         1673    646  31°26′N 100°27′W﻿ / ﻿31.44°N 100.45°W  
344         3591   1386  37°44′N 121°27′W﻿ / ﻿37.73°N 121.45°W  
345         6008   2320    26°10′N 80°16′W﻿ / ﻿26.17°N 80.26°W  

[346 rows x 10 columns]

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [16]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")


[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [19]:
URL = ("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

response = requests.get(URL, headers=HEADERS)

In [93]:
soup = BeautifulSoup(response.text, "html.parser")
# Find all course blocks
course_blocks = soup.find_all("div", "courseblock")

In [106]:
import re

rows = []

for block in course_blocks:

    course = block.find("strong")
    course_name = course.get_text(strip=True) if course else ""


    course_name = re.sub(r'\.\d+\s*units?\.?', '', course_name).strip()


    quarter = block.find("p", "noindent")
    quarter_name = quarter.get_text(strip=True) if quarter else ""


    rows.append({
        "course": course_name,
        "quarter": quarter_name
    })

In [107]:
pd.DataFrame(rows)

,course,quarter
0,DATA 100. Data Science for All I,"Term Typically Offered: F, W, SP"
1,DATA 301. Introduction to Data Science,"Term Typically Offered: F, W, SP"
2,DATA 401. Data Science Process and Ethics,Term Typically Offered: F
3,DATA 402. Mathematical Foundations of Data Sci...,Term Typically Offered: F
4,DATA 403. Data Science Projects Laboratory,Term Typically Offered: F
...,...,...
69,STAT 551. Statistical Learning with R,Term Typically Offered: F
70,STAT 566. Graduate Consulting Practicum,Term Typically Offered: SP
71,STAT 570. Selected Advanced Topics.1-4 units,Term Typically Offered: TBD
72,STAT 590. Graduate Seminar in Statistics,"Term Typically Offered: F, W, SP"


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag